### Word Group Prepare For Youtube Tool

In [72]:
import firebase_admin
from firebase_admin import credentials, firestore, storage

In [73]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [74]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [75]:
# language pair
lang_folder = "Arabic"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian

# language detail num
if lang_folder == "English":
    lang_id = "1"
elif lang_folder == "Turkish":
    lang_id = "2"
elif lang_folder == "French":
    lang_id = "3"
elif lang_folder == "Spanish":
    lang_id = "4"
elif lang_folder == "Portuguese":
    lang_id = "5"
elif lang_folder == "Dutch":
    lang_id = "6"
elif lang_folder == "Italian":
    lang_id = "7"
elif lang_folder == "German":
    lang_id = "8"
elif lang_folder == "Arabic":
    lang_id = "9"
else:
    pass

# prefix suffix file
prefix_suffix = False  # True, False  # True for adding prefix suffix word
native_word = True # True for adding native word

# adding output file extention
if prefix_suffix & native_word:
    file_ext = "5"
elif (not prefix_suffix) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              

# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"Language = {lang_folder}\nLang Id = {lang_id}\nFile Extention = {file_ext}")

Language = Italian
Lang Id = 7
File Extention = 6


In [76]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Language Level/Result/2-Word Group Prepare For Youtube Tool"

Path(path).mkdir(parents=True, exist_ok=True)

#### Read Data

In [77]:
parent_folder_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Language Level/Data/1-Word Group In Youtube Sentence"

In [78]:
df_28 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_28_Word{file_ext}.csv")
df_28 = df_28.head(700)
df_28

,search_string,start_time,end_time,sentence,video_id,video_url
0,che sono una cosa che non lo,206,208,che sono una cosa che non lo sa più,iBdSB7CjB0A,https://www.youtube.com/watch?v=iBdSB7CjB0A&t=...
1,è una cosa che non si,3373,3375,è una cosa che non si vede,0fjmn2TpzH4,https://www.youtube.com/watch?v=0fjmn2TpzH4&t=...
2,che non è una cosa da,322,323,che non è una cosa da poco,X9Z9iwK4whQ,https://www.youtube.com/watch?v=X9Z9iwK4whQ&t=...
3,con che cosa con una,80,83,con che cosa con una frusta,ZsnF4gwcYaI,https://www.youtube.com/watch?v=ZsnF4gwcYaI&t=80s
4,e non è una cosa da,210,212,e non è una cosa da poco,Efb1ZbVxHik,https://www.youtube.com/watch?v=Efb1ZbVxHik&t=...
...,...,...,...,...,...,...
614,a in,246,254,a in,6maqm10uoyQ,https://www.youtube.com/watch?v=6maqm10uoyQ&t=...
615,l a,50,58,l a,rgOXzL1GNGU,https://www.youtube.com/watch?v=rgOXzL1GNGU&t=50s
616,a i,1284,1287,a i,91TL1WhzH4c,https://www.youtube.com/watch?v=91TL1WhzH4c&t=...
617,a l,354,360,a l,91TL1WhzH4c,https://www.youtube.com/watch?v=91TL1WhzH4c&t=...


In [79]:
df_200 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_200_Word{file_ext}.csv")
df_200 = df_200.head(700)
df_200

,search_string,start_time,end_time,sentence,video_id,video_url
0,vedere meglio e siamo qui allora quello che no...,1661,1669,vedere meglio e siamo qui allora quello che no...,GH1Zq1GarM4,https://www.youtube.com/watch?v=GH1Zq1GarM4&t=...
1,come tu hai fatto sì io ho fatto questa cosa h...,1372,1387,come tu hai fatto sì io ho fatto questa cosa h...,_9o21e4PwZs,https://www.youtube.com/watch?v=_9o21e4PwZs&t=...
2,non è quello ok adesso sto a casa di qualcuno ...,36,41,probabilmente non è quello ok adesso sto a cas...,9SesLBahFjc,https://www.youtube.com/watch?v=9SesLBahFjc&t=36s
3,quello che siamo perché lo siamo già stato pri...,20,25,a capire quello che siamo perché lo siamo già ...,L2CyvarDzi8,https://www.youtube.com/watch?v=L2CyvarDzi8&t=20s
4,che tutti noi prima o poi o abbiamo fatto o ab...,107,114,have che tutti noi prima o poi o abbiamo fatto...,sHTcserYi68,https://www.youtube.com/watch?v=sHTcserYi68&t=...
...,...,...,...,...,...,...
695,niente poi dove ci sono le,1187,1189,niente poi dove ci sono le argille,IbwUzsSYMAQ,https://www.youtube.com/watch?v=IbwUzsSYMAQ&t=...
696,qui ci sta la parte quella,189,193,qui ci sta la parte quella buona,tXxGRUTyX3k,https://www.youtube.com/watch?v=tXxGRUTyX3k&t=...
697,quando vuoi sai dove siamo,1384,1387,quando vuoi sai dove siamo,JLgQbUH47Nc,https://www.youtube.com/watch?v=JLgQbUH47Nc&t=...
698,se hai mai visto o hai mai,29,30,se hai mai visto o hai mai visitato,v6x6KrBUqi0,https://www.youtube.com/watch?v=v6x6KrBUqi0&t=29s


In [80]:
df_1000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_1000_Word{file_ext}.csv")
df_1000 = df_1000.head(700)
df_1000

,search_string,start_time,end_time,sentence,video_id,video_url
0,tu hai sentito parlare di questa cosa semplice...,1469,1475,tu hai sentito parlare di questa cosa semplice...,aFjEiPXtL6w,https://www.youtube.com/watch?v=aFjEiPXtL6w&t=...
1,grazie per essere stato o per essere stata qui...,455,463,grazie per essere stato o per essere stata qui...,eljZW_3E79g,https://www.youtube.com/watch?v=eljZW_3E79g&t=...
2,perché questo video è stato molto più difficil...,58,64,perché questo video è stato molto più difficil...,brR9KopWpt4,https://www.youtube.com/watch?v=brR9KopWpt4&t=58s
3,grazie per essere stato o per essere stata qui...,494,501,grazie per essere stato o per essere stata qui...,3ArISj4cyPo,https://www.youtube.com/watch?v=3ArISj4cyPo&t=...
4,una domanda semplice questa è la nostra ultima...,637,647,buonasera corrado una domanda semplice questa ...,HAqfb5O8LaY,https://www.youtube.com/watch?v=HAqfb5O8LaY&t=...
...,...,...,...,...,...,...
695,stai pensando abbastanza dobbiamo trovare una,61,64,stai pensando abbastanza dobbiamo trovare una ...,b2iWOJumXD4,https://www.youtube.com/watch?v=b2iWOJumXD4&t=61s
696,dai ma chi ha scritto sta roba qualcuno sotto,21,24,dai ma chi ha scritto sta roba qualcuno sotto ...,OuwSObFoiqc,https://www.youtube.com/watch?v=OuwSObFoiqc&t=21s
697,molte più cose esattamente quando gli servono,587,590,molte più cose esattamente quando gli servono,YiBvqfntmXQ,https://www.youtube.com/watch?v=YiBvqfntmXQ&t=...
698,cambiare video puoi già passare ad altre cose,871,874,cambiare video puoi già passare ad altre cose,2_S1RkwiQgE,https://www.youtube.com/watch?v=2_S1RkwiQgE&t=...


In [81]:
df_5000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_5000_Word{file_ext}.csv")
df_5000 = df_5000.head(700)
df_5000

,search_string,start_time,end_time,sentence,video_id,video_url
0,ciao come stai vuol dire che del tuo umore imp...,17,38,ciao come stai vuol dire che del tuo umore imp...,8PQ71A2qPRk,https://www.youtube.com/watch?v=8PQ71A2qPRk&t=17s
1,comunque ti ringrazio per essere stato in mia ...,800,812,comunque ti ringrazio per essere stato in mia ...,uB0AorgbwTc,https://www.youtube.com/watch?v=uB0AorgbwTc&t=...
2,andati a letto i bambini dopo un ora stavano t...,250,261,andati a letto i bambini dopo un ora stavano t...,CK71slKyiXU,https://www.youtube.com/watch?v=CK71slKyiXU&t=...
3,per cui in qualche modo possiamo vedere se riu...,820,830,per cui in qualche modo possiamo vedere se riu...,Ek0vTrfaxX0,https://www.youtube.com/watch?v=Ek0vTrfaxX0&t=...
4,seguendo però sempre la sensazione che si ha n...,739,748,seguendo però sempre la sensazione che si ha n...,uB0AorgbwTc,https://www.youtube.com/watch?v=uB0AorgbwTc&t=...
...,...,...,...,...,...,...
695,prossima volta va bene mi ha fatto molto bene ...,1673,1678,prossima volta va bene mi ha fatto molto bene ...,6zySBJxlgH4,https://www.youtube.com/watch?v=6zySBJxlgH4&t=...
696,interrompere tutti continuamente quindi le fa ...,271,275,interrompere tutti continuamente quindi le fa ...,3BIx81T3AIY,https://www.youtube.com/watch?v=3BIx81T3AIY&t=...
697,questa confusione noi noi facciamo confusione ...,767,773,questa confusione noi noi facciamo confusione ...,_8cshEYEpQs,https://www.youtube.com/watch?v=_8cshEYEpQs&t=...
698,nei prossimi video spesso si chiede al candida...,498,503,approfondiremo nei prossimi video spesso si ch...,Y8J22AnqIFI,https://www.youtube.com/watch?v=Y8J22AnqIFI&t=...


In [82]:
df_10000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_10000_Word{file_ext}.csv")
df_10000 = df_10000.head(700)
df_10000

,search_string,start_time,end_time,sentence,video_id,video_url
0,quindi venivo qui due volte a settimana il sab...,294,305,quindi venivo qui due volte a settimana il sab...,miyE4qxnx70,https://www.youtube.com/watch?v=miyE4qxnx70&t=...
1,quando mi è stato chiesto di partecipare a que...,5,13,quando mi è stato chiesto di partecipare a que...,WwxadFn4iV8,https://www.youtube.com/watch?v=WwxadFn4iV8&t=5s
2,sarà necessaria una preparazione quasi formale...,223,241,sarà necessaria una preparazione quasi formale...,ldpgAhQtANs,https://www.youtube.com/watch?v=ldpgAhQtANs&t=...
3,finale molto essenziale molto semplice ma cari...,1018,1030,finale molto essenziale molto semplice ma cari...,u6LDbl4fZMk,https://www.youtube.com/watch?v=u6LDbl4fZMk&t=...
4,la cosa interessante che si può vedere dalle f...,204,215,la cosa interessante che si può vedere dalle f...,yqH9YdpZihk,https://www.youtube.com/watch?v=yqH9YdpZihk&t=...
...,...,...,...,...,...,...
695,visto prima amore niente papà antonio ti devo ...,737,744,visto prima amore niente papà antonio ti devo ...,nTwxFYZW0E4,https://www.youtube.com/watch?v=nTwxFYZW0E4&t=...
696,comune azioni ottenere prodotti dagli animali ...,1246,1253,nellarea comune azioni ottenere prodotti dagli...,egrAt4prbl0,https://www.youtube.com/watch?v=egrAt4prbl0&t=...
697,boss mi dispiace tantissimo zombie infatti gli...,1379,1385,boss mi dispiace tantissimo zombie infatti gli...,GAqADZjN1f0,https://www.youtube.com/watch?v=GAqADZjN1f0&t=...
698,ve lo faccio vedere quando lo completato allor...,6165,6173,ve lo faccio vedere quando lo completato allor...,y6b1NXprZts,https://www.youtube.com/watch?v=y6b1NXprZts&t=...


In [83]:
df_20000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_20000_Word{file_ext}.csv")
df_20000 = df_20000.head(700)
df_20000

,search_string,start_time,end_time,sentence,video_id,video_url
0,la manifestazione è finita prima del previsto ...,61,63,la manifestazione è finita prima del previsto ...,b5q9uBE5H_s,https://www.youtube.com/watch?v=b5q9uBE5H_s&t=61s
1,finale molto essenziale molto semplice ma cari...,1018,1031,finale molto essenziale molto semplice ma cari...,u6LDbl4fZMk,https://www.youtube.com/watch?v=u6LDbl4fZMk&t=...
2,che stanno proprio nel nostro quartiere verame...,407,420,che stanno proprio nel nostro quartiere verame...,CK71slKyiXU,https://www.youtube.com/watch?v=CK71slKyiXU&t=...
3,ciao cari amici benvenuti nella mia cucina ogg...,0,9,ciao cari amici benvenuti nella mia cucina ogg...,AkW2coApj00,https://www.youtube.com/watch?v=AkW2coApj00&t=0s
4,quello che ci chiedete forse dal primo giorno ...,36,47,quello che ci chiedete forse dal primo giorno ...,9HuoF2bl2us,https://www.youtube.com/watch?v=9HuoF2bl2us&t=36s
...,...,...,...,...,...,...
695,ragazzi arriviamo al punto della sospensione d...,608,612,ragazzi arriviamo al punto della sospensione d...,op9ST8Se2Z8,https://www.youtube.com/watch?v=op9ST8Se2Z8&t=...
696,dettagli poi qui su sottotitoli allora questa ...,135,147,dettagli poi qui su sottotitoli allora questa ...,7sH07JoqCeo,https://www.youtube.com/watch?v=7sH07JoqCeo&t=...
697,prendere questa carta qua ora ne rimangono due...,791,798,prendere questa carta qua ora ne rimangono due...,A8QYA2EH8Xg,https://www.youtube.com/watch?v=A8QYA2EH8Xg&t=...
698,vasto però vogliamo aggiungere ancora qualcosa...,479,484,vasto però vogliamo aggiungere ancora qualcosa...,XFNzWC4nAm0,https://www.youtube.com/watch?v=XFNzWC4nAm0&t=...


In [84]:
df_40000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_40000_Word{file_ext}.csv")
df_40000 = df_40000.head(700)
df_40000

,search_string,start_time,end_time,sentence,video_id,video_url
0,si racconta che dopo una discussione sugli aff...,564,569,si racconta che dopo una discussione sugli aff...,2HVKJg__wYk,https://www.youtube.com/watch?v=2HVKJg__wYk&t=...
1,distanza da non essere a rischio ma abbastanza...,549,563,a debita distanza da non essere a rischio ma a...,fomcMM1XA5A,https://www.youtube.com/watch?v=fomcMM1XA5A&t=...
2,quindi ci rendiamo conto che la storia poi va ...,645,668,quindi ci rendiamo conto che la storia poi va ...,dMNIjzeZA1c,https://www.youtube.com/watch?v=dMNIjzeZA1c&t=...
3,consiglio di utilizzare il mix africano per un...,42,46,consiglio di utilizzare il mix africano per un...,AwdMIo8Lv5U,https://www.youtube.com/watch?v=AwdMIo8Lv5U&t=42s
4,la manifestazione è finita prima del previsto ...,61,63,la manifestazione è finita prima del previsto ...,b5q9uBE5H_s,https://www.youtube.com/watch?v=b5q9uBE5H_s&t=61s
...,...,...,...,...,...,...
695,dispiace davvero tanto come posso rimediare fa...,74,80,dispiace davvero tanto come posso rimediare fa...,N39wvHqh1qU,https://www.youtube.com/watch?v=N39wvHqh1qU&t=74s
696,iniziare ok allora prima cosa dovete spostarvi...,4130,4142,iniziare ok allora prima cosa dovete spostarvi...,FIgq4DFtUSI,https://www.youtube.com/watch?v=FIgq4DFtUSI&t=...
697,modo più informale scrivendo facendo informazi...,125,130,modo più informale scrivendo facendo informazi...,YR1RNnIp7jQ,https://www.youtube.com/watch?v=YR1RNnIp7jQ&t=...
698,mille cose mille progetti ci siamo trovati sul...,791,798,mille cose mille progetti ci siamo trovati sul...,OkVaUOGl8o4,https://www.youtube.com/watch?v=OkVaUOGl8o4&t=...


#### End Time Reassign

In [85]:
df_28_reassign = df_28.copy() # important because when it doesn't copy, main df are also change.
df_200_reassign = df_200.copy()
df_1000_reassign = df_1000.copy()
df_5000_reassign = df_5000.copy()
df_10000_reassign = df_10000.copy()
df_20000_reassign = df_20000.copy()
df_40000_reassign = df_40000.copy()

In [86]:
df_28_reassign["end_time"] = df_28_reassign["start_time"] + 1
df_200_reassign["end_time"] = df_200_reassign["start_time"] + 1
df_1000_reassign["end_time"] = df_1000_reassign["start_time"] + 1
df_5000_reassign["end_time"] = df_5000_reassign["start_time"] + 1
df_10000_reassign["end_time"] = df_10000_reassign["start_time"] + 1
df_20000_reassign["end_time"] = df_20000_reassign["start_time"] + 1
df_40000_reassign["end_time"] = df_40000_reassign["start_time"] + 1

In [87]:
out_path = "/home/kurubal/Downloads"

In [50]:
df_28_reassign.to_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_28_Word{file_ext}.xlsx", index=False)
df_200_reassign.to_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_200_Word{file_ext}.xlsx", index=False)
df_1000_reassign.to_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_1000_Word{file_ext}.xlsx", index=False)
df_5000_reassign.to_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_5000_Word{file_ext}.xlsx", index=False)
df_10000_reassign.to_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_10000_Word{file_ext}.xlsx", index=False)
df_20000_reassign.to_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_20000_Word{file_ext}.xlsx", index=False)
df_40000_reassign.to_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_40000_Word{file_ext}.xlsx", index=False)

In [ ]:
# !!!!! End Time Reassign files result check on youtube tool and change file with youtube tool file result as same name (old file should remove) !!!!! 

#### End Time Reassign Convert Orjinal (From Youtube Tool Result)

In [88]:
df_28_youtube_tool = pd.read_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_28_Word{file_ext}.xlsx")
df_28_video_list = list(set(df_28_youtube_tool["video_id"]))

In [89]:
df_200_youtube_tool = pd.read_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_200_Word{file_ext}.xlsx")
df_200_video_list = list(set(df_200_youtube_tool["video_id"]))

In [90]:
df_1000_youtube_tool = pd.read_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_1000_Word{file_ext}.xlsx")
df_1000_video_list = list(set(df_1000_youtube_tool["video_id"]))

In [91]:
df_5000_youtube_tool = pd.read_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_5000_Word{file_ext}.xlsx")
df_5000_video_list = list(set(df_5000_youtube_tool["video_id"]))

In [92]:
df_10000_youtube_tool = pd.read_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_10000_Word{file_ext}.xlsx")
df_10000_video_list = list(set(df_10000_youtube_tool["video_id"]))

In [93]:
df_20000_youtube_tool = pd.read_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_20000_Word{file_ext}.xlsx")
df_20000_video_list = list(set(df_20000_youtube_tool["video_id"]))

In [94]:
df_40000_youtube_tool = pd.read_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_40000_Word{file_ext}.xlsx")
df_40000_video_list = list(set(df_40000_youtube_tool["video_id"]))

##### Video List In video_id 

In [95]:
df_28_new = df_28[df_28["video_id"].isin(df_28_video_list)]
#df_28_new.to_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_28_Word{file_ext}.csv", index=False)
df_28_new.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_28_Word{file_ext}.xlsx", index=False)
df_28_new

,search_string,start_time,end_time,sentence,video_id,video_url
0,che sono una cosa che non lo,206,208,che sono una cosa che non lo sa più,iBdSB7CjB0A,https://www.youtube.com/watch?v=iBdSB7CjB0A&t=...
1,è una cosa che non si,3373,3375,è una cosa che non si vede,0fjmn2TpzH4,https://www.youtube.com/watch?v=0fjmn2TpzH4&t=...
2,che non è una cosa da,322,323,che non è una cosa da poco,X9Z9iwK4whQ,https://www.youtube.com/watch?v=X9Z9iwK4whQ&t=...
3,con che cosa con una,80,83,con che cosa con una frusta,ZsnF4gwcYaI,https://www.youtube.com/watch?v=ZsnF4gwcYaI&t=80s
4,e non è una cosa da,210,212,e non è una cosa da poco,Efb1ZbVxHik,https://www.youtube.com/watch?v=Efb1ZbVxHik&t=...
...,...,...,...,...,...,...
614,a in,246,254,a in,6maqm10uoyQ,https://www.youtube.com/watch?v=6maqm10uoyQ&t=...
615,l a,50,58,l a,rgOXzL1GNGU,https://www.youtube.com/watch?v=rgOXzL1GNGU&t=50s
616,a i,1284,1287,a i,91TL1WhzH4c,https://www.youtube.com/watch?v=91TL1WhzH4c&t=...
617,a l,354,360,a l,91TL1WhzH4c,https://www.youtube.com/watch?v=91TL1WhzH4c&t=...


In [96]:
df_200_new = df_200[df_200["video_id"].isin(df_200_video_list)]
#df_200_new.to_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_200_Word{file_ext}.csv", index=False)
df_200_new.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_200_Word{file_ext}.xlsx", index=False)
df_200_new

,search_string,start_time,end_time,sentence,video_id,video_url
0,vedere meglio e siamo qui allora quello che no...,1661,1669,vedere meglio e siamo qui allora quello che no...,GH1Zq1GarM4,https://www.youtube.com/watch?v=GH1Zq1GarM4&t=...
1,come tu hai fatto sì io ho fatto questa cosa h...,1372,1387,come tu hai fatto sì io ho fatto questa cosa h...,_9o21e4PwZs,https://www.youtube.com/watch?v=_9o21e4PwZs&t=...
2,non è quello ok adesso sto a casa di qualcuno ...,36,41,probabilmente non è quello ok adesso sto a cas...,9SesLBahFjc,https://www.youtube.com/watch?v=9SesLBahFjc&t=36s
3,quello che siamo perché lo siamo già stato pri...,20,25,a capire quello che siamo perché lo siamo già ...,L2CyvarDzi8,https://www.youtube.com/watch?v=L2CyvarDzi8&t=20s
4,che tutti noi prima o poi o abbiamo fatto o ab...,107,114,have che tutti noi prima o poi o abbiamo fatto...,sHTcserYi68,https://www.youtube.com/watch?v=sHTcserYi68&t=...
...,...,...,...,...,...,...
695,niente poi dove ci sono le,1187,1189,niente poi dove ci sono le argille,IbwUzsSYMAQ,https://www.youtube.com/watch?v=IbwUzsSYMAQ&t=...
696,qui ci sta la parte quella,189,193,qui ci sta la parte quella buona,tXxGRUTyX3k,https://www.youtube.com/watch?v=tXxGRUTyX3k&t=...
697,quando vuoi sai dove siamo,1384,1387,quando vuoi sai dove siamo,JLgQbUH47Nc,https://www.youtube.com/watch?v=JLgQbUH47Nc&t=...
698,se hai mai visto o hai mai,29,30,se hai mai visto o hai mai visitato,v6x6KrBUqi0,https://www.youtube.com/watch?v=v6x6KrBUqi0&t=29s


In [97]:
df_1000_new = df_1000[df_1000["video_id"].isin(df_1000_video_list)]
#df_1000_new.to_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_1000_Word{file_ext}.csv", index=False)
df_1000_new.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_1000_Word{file_ext}.xlsx", index=False)
df_1000_new

,search_string,start_time,end_time,sentence,video_id,video_url
0,tu hai sentito parlare di questa cosa semplice...,1469,1475,tu hai sentito parlare di questa cosa semplice...,aFjEiPXtL6w,https://www.youtube.com/watch?v=aFjEiPXtL6w&t=...
1,grazie per essere stato o per essere stata qui...,455,463,grazie per essere stato o per essere stata qui...,eljZW_3E79g,https://www.youtube.com/watch?v=eljZW_3E79g&t=...
2,perché questo video è stato molto più difficil...,58,64,perché questo video è stato molto più difficil...,brR9KopWpt4,https://www.youtube.com/watch?v=brR9KopWpt4&t=58s
3,grazie per essere stato o per essere stata qui...,494,501,grazie per essere stato o per essere stata qui...,3ArISj4cyPo,https://www.youtube.com/watch?v=3ArISj4cyPo&t=...
4,una domanda semplice questa è la nostra ultima...,637,647,buonasera corrado una domanda semplice questa ...,HAqfb5O8LaY,https://www.youtube.com/watch?v=HAqfb5O8LaY&t=...
...,...,...,...,...,...,...
695,stai pensando abbastanza dobbiamo trovare una,61,64,stai pensando abbastanza dobbiamo trovare una ...,b2iWOJumXD4,https://www.youtube.com/watch?v=b2iWOJumXD4&t=61s
696,dai ma chi ha scritto sta roba qualcuno sotto,21,24,dai ma chi ha scritto sta roba qualcuno sotto ...,OuwSObFoiqc,https://www.youtube.com/watch?v=OuwSObFoiqc&t=21s
697,molte più cose esattamente quando gli servono,587,590,molte più cose esattamente quando gli servono,YiBvqfntmXQ,https://www.youtube.com/watch?v=YiBvqfntmXQ&t=...
698,cambiare video puoi già passare ad altre cose,871,874,cambiare video puoi già passare ad altre cose,2_S1RkwiQgE,https://www.youtube.com/watch?v=2_S1RkwiQgE&t=...


In [98]:
df_5000_new = df_5000[df_5000["video_id"].isin(df_5000_video_list)]
#df_5000_new.to_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_5000_Word{file_ext}.csv", index=False)
df_5000_new.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_5000_Word{file_ext}.xlsx", index=False)
df_5000_new

,search_string,start_time,end_time,sentence,video_id,video_url
0,ciao come stai vuol dire che del tuo umore imp...,17,38,ciao come stai vuol dire che del tuo umore imp...,8PQ71A2qPRk,https://www.youtube.com/watch?v=8PQ71A2qPRk&t=17s
1,comunque ti ringrazio per essere stato in mia ...,800,812,comunque ti ringrazio per essere stato in mia ...,uB0AorgbwTc,https://www.youtube.com/watch?v=uB0AorgbwTc&t=...
2,andati a letto i bambini dopo un ora stavano t...,250,261,andati a letto i bambini dopo un ora stavano t...,CK71slKyiXU,https://www.youtube.com/watch?v=CK71slKyiXU&t=...
3,per cui in qualche modo possiamo vedere se riu...,820,830,per cui in qualche modo possiamo vedere se riu...,Ek0vTrfaxX0,https://www.youtube.com/watch?v=Ek0vTrfaxX0&t=...
4,seguendo però sempre la sensazione che si ha n...,739,748,seguendo però sempre la sensazione che si ha n...,uB0AorgbwTc,https://www.youtube.com/watch?v=uB0AorgbwTc&t=...
...,...,...,...,...,...,...
695,prossima volta va bene mi ha fatto molto bene ...,1673,1678,prossima volta va bene mi ha fatto molto bene ...,6zySBJxlgH4,https://www.youtube.com/watch?v=6zySBJxlgH4&t=...
696,interrompere tutti continuamente quindi le fa ...,271,275,interrompere tutti continuamente quindi le fa ...,3BIx81T3AIY,https://www.youtube.com/watch?v=3BIx81T3AIY&t=...
697,questa confusione noi noi facciamo confusione ...,767,773,questa confusione noi noi facciamo confusione ...,_8cshEYEpQs,https://www.youtube.com/watch?v=_8cshEYEpQs&t=...
698,nei prossimi video spesso si chiede al candida...,498,503,approfondiremo nei prossimi video spesso si ch...,Y8J22AnqIFI,https://www.youtube.com/watch?v=Y8J22AnqIFI&t=...


In [99]:
df_10000_new = df_10000[df_10000["video_id"].isin(df_10000_video_list)]
#df_10000_new.to_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_10000_Word{file_ext}.csv", index=False)
df_10000_new.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_10000_Word{file_ext}.xlsx", index=False)
df_10000_new

,search_string,start_time,end_time,sentence,video_id,video_url
0,quindi venivo qui due volte a settimana il sab...,294,305,quindi venivo qui due volte a settimana il sab...,miyE4qxnx70,https://www.youtube.com/watch?v=miyE4qxnx70&t=...
1,quando mi è stato chiesto di partecipare a que...,5,13,quando mi è stato chiesto di partecipare a que...,WwxadFn4iV8,https://www.youtube.com/watch?v=WwxadFn4iV8&t=5s
2,sarà necessaria una preparazione quasi formale...,223,241,sarà necessaria una preparazione quasi formale...,ldpgAhQtANs,https://www.youtube.com/watch?v=ldpgAhQtANs&t=...
3,finale molto essenziale molto semplice ma cari...,1018,1030,finale molto essenziale molto semplice ma cari...,u6LDbl4fZMk,https://www.youtube.com/watch?v=u6LDbl4fZMk&t=...
4,la cosa interessante che si può vedere dalle f...,204,215,la cosa interessante che si può vedere dalle f...,yqH9YdpZihk,https://www.youtube.com/watch?v=yqH9YdpZihk&t=...
...,...,...,...,...,...,...
695,visto prima amore niente papà antonio ti devo ...,737,744,visto prima amore niente papà antonio ti devo ...,nTwxFYZW0E4,https://www.youtube.com/watch?v=nTwxFYZW0E4&t=...
696,comune azioni ottenere prodotti dagli animali ...,1246,1253,nellarea comune azioni ottenere prodotti dagli...,egrAt4prbl0,https://www.youtube.com/watch?v=egrAt4prbl0&t=...
697,boss mi dispiace tantissimo zombie infatti gli...,1379,1385,boss mi dispiace tantissimo zombie infatti gli...,GAqADZjN1f0,https://www.youtube.com/watch?v=GAqADZjN1f0&t=...
698,ve lo faccio vedere quando lo completato allor...,6165,6173,ve lo faccio vedere quando lo completato allor...,y6b1NXprZts,https://www.youtube.com/watch?v=y6b1NXprZts&t=...


In [100]:
df_20000_new = df_20000[df_20000["video_id"].isin(df_20000_video_list)]
#df_20000_new.to_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_20000_Word{file_ext}.csv", index=False)
df_20000_new.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_20000_Word{file_ext}.xlsx", index=False)
df_20000_new

,search_string,start_time,end_time,sentence,video_id,video_url
0,la manifestazione è finita prima del previsto ...,61,63,la manifestazione è finita prima del previsto ...,b5q9uBE5H_s,https://www.youtube.com/watch?v=b5q9uBE5H_s&t=61s
1,finale molto essenziale molto semplice ma cari...,1018,1031,finale molto essenziale molto semplice ma cari...,u6LDbl4fZMk,https://www.youtube.com/watch?v=u6LDbl4fZMk&t=...
2,che stanno proprio nel nostro quartiere verame...,407,420,che stanno proprio nel nostro quartiere verame...,CK71slKyiXU,https://www.youtube.com/watch?v=CK71slKyiXU&t=...
3,ciao cari amici benvenuti nella mia cucina ogg...,0,9,ciao cari amici benvenuti nella mia cucina ogg...,AkW2coApj00,https://www.youtube.com/watch?v=AkW2coApj00&t=0s
4,quello che ci chiedete forse dal primo giorno ...,36,47,quello che ci chiedete forse dal primo giorno ...,9HuoF2bl2us,https://www.youtube.com/watch?v=9HuoF2bl2us&t=36s
...,...,...,...,...,...,...
695,ragazzi arriviamo al punto della sospensione d...,608,612,ragazzi arriviamo al punto della sospensione d...,op9ST8Se2Z8,https://www.youtube.com/watch?v=op9ST8Se2Z8&t=...
696,dettagli poi qui su sottotitoli allora questa ...,135,147,dettagli poi qui su sottotitoli allora questa ...,7sH07JoqCeo,https://www.youtube.com/watch?v=7sH07JoqCeo&t=...
697,prendere questa carta qua ora ne rimangono due...,791,798,prendere questa carta qua ora ne rimangono due...,A8QYA2EH8Xg,https://www.youtube.com/watch?v=A8QYA2EH8Xg&t=...
698,vasto però vogliamo aggiungere ancora qualcosa...,479,484,vasto però vogliamo aggiungere ancora qualcosa...,XFNzWC4nAm0,https://www.youtube.com/watch?v=XFNzWC4nAm0&t=...


In [101]:
df_40000_new = df_40000[df_40000["video_id"].isin(df_40000_video_list)]
#df_40000_new.to_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_40000_Word{file_ext}.csv", index=False)
df_40000_new.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_40000_Word{file_ext}.xlsx", index=False)
df_40000_new

,search_string,start_time,end_time,sentence,video_id,video_url
0,si racconta che dopo una discussione sugli aff...,564,569,si racconta che dopo una discussione sugli aff...,2HVKJg__wYk,https://www.youtube.com/watch?v=2HVKJg__wYk&t=...
1,distanza da non essere a rischio ma abbastanza...,549,563,a debita distanza da non essere a rischio ma a...,fomcMM1XA5A,https://www.youtube.com/watch?v=fomcMM1XA5A&t=...
2,quindi ci rendiamo conto che la storia poi va ...,645,668,quindi ci rendiamo conto che la storia poi va ...,dMNIjzeZA1c,https://www.youtube.com/watch?v=dMNIjzeZA1c&t=...
3,consiglio di utilizzare il mix africano per un...,42,46,consiglio di utilizzare il mix africano per un...,AwdMIo8Lv5U,https://www.youtube.com/watch?v=AwdMIo8Lv5U&t=42s
4,la manifestazione è finita prima del previsto ...,61,63,la manifestazione è finita prima del previsto ...,b5q9uBE5H_s,https://www.youtube.com/watch?v=b5q9uBE5H_s&t=61s
...,...,...,...,...,...,...
695,dispiace davvero tanto come posso rimediare fa...,74,80,dispiace davvero tanto come posso rimediare fa...,N39wvHqh1qU,https://www.youtube.com/watch?v=N39wvHqh1qU&t=74s
696,iniziare ok allora prima cosa dovete spostarvi...,4130,4142,iniziare ok allora prima cosa dovete spostarvi...,FIgq4DFtUSI,https://www.youtube.com/watch?v=FIgq4DFtUSI&t=...
697,modo più informale scrivendo facendo informazi...,125,130,modo più informale scrivendo facendo informazi...,YR1RNnIp7jQ,https://www.youtube.com/watch?v=YR1RNnIp7jQ&t=...
698,mille cose mille progetti ci siamo trovati sul...,791,798,mille cose mille progetti ci siamo trovati sul...,OkVaUOGl8o4,https://www.youtube.com/watch?v=OkVaUOGl8o4&t=...


#### Copy Move And Delete

In [102]:
output_file2 = glob.glob(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_*_Word{file_ext}.*")
output_file2

['Italian_Word_Group_Max_In_Youtube_Sentence_28_Word6.xlsx',
 'Italian_Word_Group_Max_In_Youtube_Sentence_200_Word6.xlsx',
 'Italian_Word_Group_Max_In_Youtube_Sentence_1000_Word6.xlsx',
 'Italian_Word_Group_Max_In_Youtube_Sentence_5000_Word6.xlsx',
 'Italian_Word_Group_Max_In_Youtube_Sentence_10000_Word6.xlsx',
 'Italian_Word_Group_Max_In_Youtube_Sentence_20000_Word6.xlsx',
 'Italian_Word_Group_Max_In_Youtube_Sentence_40000_Word6.xlsx']

In [103]:
for l in output_file2:
    source = l # source directory
    destination = path
    shutil.copy2(source, destination)

In [104]:
for j in output_file2:
    try:
        os.remove(j)
    except:
        pass